In [21]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

def calculate_atr(df, period):
    """Calculate Average True Range (ATR)"""
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1)
    true_range = tr.max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr

def get_supertrend(df, period=9, multiplier=2):
    """Calculate Supertrend indicator"""
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Calculate ATR
    df['ATR'] = calculate_atr(df, period)
    
    # Calculate basic upper and lower bands
    df['HL2'] = (df['High'] + df['Low']) / 2  # Add HL2 as a separate column
    df['Basic_Upper'] = df['HL2'] + (multiplier * df['ATR'])
    df['Basic_Lower'] = df['HL2'] - (multiplier * df['ATR'])
    
    # Initialize Supertrend columns with NaN
    df['Final_Upper'] = np.nan
    df['Final_Lower'] = np.nan
    df['Supertrend'] = np.nan
    df['Trend'] = 0
    
    # Set initial values
    df.loc[df.index[0], 'Final_Upper'] = df.loc[df.index[0], 'Basic_Upper']
    df.loc[df.index[0], 'Final_Lower'] = df.loc[df.index[0], 'Basic_Lower']
    df.loc[df.index[0], 'Supertrend'] = df.loc[df.index[0], 'Basic_Upper']
    df.loc[df.index[0], 'Trend'] = 1
    
    # Calculate Final Upper/Lower bands and Supertrend
    for i in range(1, len(df)):
        curr_idx = df.index[i]
        prev_idx = df.index[i-1]
        
        # Final Upper Band
        if (df.loc[curr_idx, 'Basic_Upper'] < df.loc[prev_idx, 'Final_Upper'] or 
            df.loc[prev_idx, 'Close'] > df.loc[prev_idx, 'Final_Upper']):
            df.loc[curr_idx, 'Final_Upper'] = df.loc[curr_idx, 'Basic_Upper']
        else:
            df.loc[curr_idx, 'Final_Upper'] = df.loc[prev_idx, 'Final_Upper']
            
        # Final Lower Band
        if (df.loc[curr_idx, 'Basic_Lower'] > df.loc[prev_idx, 'Final_Lower'] or 
            df.loc[prev_idx, 'Close'] < df.loc[prev_idx, 'Final_Lower']):
            df.loc[curr_idx, 'Final_Lower'] = df.loc[curr_idx, 'Basic_Lower']
        else:
            df.loc[curr_idx, 'Final_Lower'] = df.loc[prev_idx, 'Final_Lower']
            
        # Supertrend
        if df.loc[prev_idx, 'Supertrend'] == df.loc[prev_idx, 'Final_Upper']:
            if df.loc[curr_idx, 'Close'] > df.loc[curr_idx, 'Final_Upper']:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Lower']
                df.loc[curr_idx, 'Trend'] = 1
            else:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Upper']
                df.loc[curr_idx, 'Trend'] = -1
        else:
            if df.loc[curr_idx, 'Close'] < df.loc[curr_idx, 'Final_Lower']:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Upper']
                df.loc[curr_idx, 'Trend'] = -1
            else:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Lower']
                df.loc[curr_idx, 'Trend'] = 1
    
    return df

def fetch_stock_data(symbol, start_date, end_date, interval='1wk'):
    """Fetch stock data from Yahoo Finance"""
    try:
        stock = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        if stock.empty:
            raise Exception(f"No data found for {symbol}")
        return stock
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def analyze_supertrend(symbol, period=9, multiplier=2):
    """Analyze stock using Supertrend indicator"""
    # Calculate date range for 2 years of weekly data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=2*365)
    
    # Fetch data
    df = fetch_stock_data(symbol, start_date, end_date)
    if df is None:
        return
    
    # Calculate Supertrend
    result = get_supertrend(df, period=period, multiplier=multiplier)
    
    # Generate signals
    result['Signal'] = np.where(result['Trend'] == 1, 'BUY', 'SELL')
    
    # Print analysis
    print(f"\nSupertrend Analysis for {symbol}")
    print("=" * 50)
    
    # Print last 10 weeks of data
    recent_data = result.tail(10)
    for idx, row in recent_data.iterrows():
        print(f"\nDate: {idx.strftime('%Y-%m-%d')}")
        print(f"Close: ${row['Close']:.2f}")
        print(f"Supertrend: ${row['Supertrend']:.2f}")
        print(f"Signal: {row['Signal']}")
        if row['Trend'] == 1:
            print("Trend: Uptrend 🔼")
        else:
            print("Trend: Downtrend 🔽")
        print("-" * 30)
    
    return result

def main():
    # Example usage
    symbols = ['AAPL', 'MSFT', 'GOOGL']

In [23]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

def calculate_atr(df, period):
    """Calculate Average True Range (ATR)"""
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1)
    true_range = tr.max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr

def get_supertrend(df, period=9, multiplier=2):
    """Calculate Supertrend indicator"""
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Calculate ATR
    df['ATR'] = calculate_atr(df, period)
    
    # Calculate basic upper and lower bands
    df['HL2'] = (df['High'] + df['Low']) / 2
    df['Basic_Upper'] = df['HL2'] + (multiplier * df['ATR'])
    df['Basic_Lower'] = df['HL2'] - (multiplier * df['ATR'])
    
    # Initialize Supertrend columns with NaN
    df['Final_Upper'] = np.nan
    df['Final_Lower'] = np.nan
    df['Supertrend'] = np.nan
    df['Trend'] = 0
    
    # Set initial values
    df.loc[df.index[0], 'Final_Upper'] = df.loc[df.index[0], 'Basic_Upper']
    df.loc[df.index[0], 'Final_Lower'] = df.loc[df.index[0], 'Basic_Lower']
    df.loc[df.index[0], 'Supertrend'] = df.loc[df.index[0], 'Basic_Upper']
    df.loc[df.index[0], 'Trend'] = 1
    
    # Calculate Final Upper/Lower bands and Supertrend
    for i in range(1, len(df)):
        curr_idx = df.index[i]
        prev_idx = df.index[i-1]
        
        curr_basic_upper = df.loc[curr_idx, 'Basic_Upper']
        curr_basic_lower = df.loc[curr_idx, 'Basic_Lower']
        curr_close = df.loc[curr_idx, 'Close']
        prev_final_upper = df.loc[prev_idx, 'Final_Upper']
        prev_final_lower = df.loc[prev_idx, 'Final_Lower']
        prev_supertrend = df.loc[prev_idx, 'Supertrend']
        prev_close = df.loc[prev_idx, 'Close']
        
        # Final Upper Band
        if (curr_basic_upper < prev_final_upper or 
            prev_close > prev_final_upper):
            df.loc[curr_idx, 'Final_Upper'] = curr_basic_upper
        else:
            df.loc[curr_idx, 'Final_Upper'] = prev_final_upper
            
        # Final Lower Band
        if (curr_basic_lower > prev_final_lower or 
            prev_close < prev_final_lower):
            df.loc[curr_idx, 'Final_Lower'] = curr_basic_lower
        else:
            df.loc[curr_idx, 'Final_Lower'] = prev_final_lower
            
        # Supertrend
        if prev_supertrend == prev_final_upper:
            if curr_close > df.loc[curr_idx, 'Final_Upper']:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Lower']
                df.loc[curr_idx, 'Trend'] = 1
            else:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Upper']
                df.loc[curr_idx, 'Trend'] = -1
        else:
            if curr_close < df.loc[curr_idx, 'Final_Lower']:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Upper']
                df.loc[curr_idx, 'Trend'] = -1
            else:
                df.loc[curr_idx, 'Supertrend'] = df.loc[curr_idx, 'Final_Lower']
                df.loc[curr_idx, 'Trend'] = 1
    
    return df

def fetch_stock_data(symbol, start_date, end_date, interval='1wk'):
    """Fetch stock data from Yahoo Finance"""
    try:
        stock = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        if stock.empty:
            raise Exception(f"No data found for {symbol}")
        return stock
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def analyze_supertrend(symbol, period=9, multiplier=2):
    """Analyze stock using Supertrend indicator"""
    # Calculate date range for 2 years of weekly data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=2*365)
    
    # Fetch data
    df = fetch_stock_data(symbol, start_date, end_date)
    if df is None:
        return
    
    # Calculate Supertrend
    result = get_supertrend(df, period=period, multiplier=multiplier)
    
    # Generate signals
    result['Signal'] = np.where(result['Trend'] == 1, 'BUY', 'SELL')
    
    # Print analysis
    print(f"\nSupertrend Analysis for {symbol}")
    print("=" * 50)
    
    # Print last 10 weeks of data
    recent_data = result.tail(10)
    for idx, row in recent_data.iterrows():
        print(f"\nDate: {idx.strftime('%Y-%m-%d')}")
        print(f"Close: ${row['Close']:.2f}")
        print(f"Supertrend: ${row['Supertrend']:.2f}")
        print(f"Signal: {row['Signal']}")
        if row['Trend'] == 1:
            print("Trend: Uptrend 🔼")
        else:
            print("Trend: Downtrend 🔽")
        print("-" * 30)
    
    return result

def main():
    # Example usage
    symbols = ['AAPL', 'MSFT', 'GOOGL']
    
    results = {}
    for symbol in symbols:
        print(f"\nAnalyzing {symbol}...")
        result = analyze_supertrend(symbol, period=9, multiplier=2)
        
        if result is not None:
            results[symbol] = result
            
            # Calculate current position
            current_data = result.iloc[-1]
            print(f"\nCurrent Status for {symbol}:")
            print(f"Current Close: ${current_data['Close']:.2f}")
            print(f"Current Supertrend: ${current_data['Supertrend']:.2f}")
            print(f"Current Signal: {current_data['Signal']}")
            print(f"Trend Direction: {'Uptrend 🔼' if current_data['Trend'] == 1 else 'Downtrend 🔽'}")
    
    return results

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed


Analyzing AAPL...
YF.download() has changed argument auto_adjust default to True


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

def calculate_atr(df, period):
    """Calculate Average True Range (ATR)"""
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1)
    true_range = tr.max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr

def get_supertrend(df, period=9, multiplier=2):
    """Calculate Supertrend indicator"""
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Calculate ATR
    df['ATR'] = calculate_atr(df, period)
    
    # Calculate basic upper and lower bands
    df['HL2'] = (df['High'] + df['Low']) / 2
    df['Basic_Upper'] = df['HL2'] + (multiplier * df['ATR'])
    df['Basic_Lower'] = df['HL2'] - (multiplier * df['ATR'])
    
    # Initialize Supertrend columns with NaN
    df['Final_Upper'] = np.nan
    df['Final_Lower'] = np.nan
    df['Supertrend'] = np.nan
    df['Trend'] = 0
    
    # Set initial values
    df.loc[df.index[0], 'Final_Upper'] = df.loc[df.index[0], 'Basic_Upper']
    df.loc[df.index[0], 'Final_Lower'] = df.loc[df.index[0], 'Basic_Lower']
    df.loc[df.index[0], 'Supertrend'] = df.loc[df.index[0], 'Basic_Upper']
    df.loc[df.index[0], 'Trend'] = 1
    
    # Calculate Final Upper/Lower bands and Supertrend
    for i in range(1, len(df)):
        curr_idx = df.index[i]
        prev_idx = df.index[i-1]
        
        # Extract scalar values using .item()
        curr_basic_upper = df.loc[curr_idx, 'Basic_Upper'].item()
        curr_basic_lower = df.loc[curr_idx, 'Basic_Lower'].item()
        curr_close = df.loc[curr_idx, 'Close'].item()
        curr_final_upper = df.loc[curr_idx, 'Final_Upper'] if i == 1 else df.loc[curr_idx, 'Final_Upper']
        curr_final_lower = df.loc[curr_idx, 'Final_Lower'] if i == 1 else df.loc[curr_idx, 'Final_Lower']
        prev_final_upper = df.loc[prev_idx, 'Final_Upper'].item()
        prev_final_lower = df.loc[prev_idx, 'Final_Lower'].item()
        prev_supertrend = df.loc[prev_idx, 'Supertrend'].item()
        prev_close = df.loc[prev_idx, 'Close'].item()
        
        # Final Upper Band
        if (curr_basic_upper < prev_final_upper or 
            prev_close > prev_final_upper):
            df.loc[curr_idx, 'Final_Upper'] = curr_basic_upper
        else:
            df.loc[curr_idx, 'Final_Upper'] = prev_final_upper
            
        # Final Lower Band
        if (curr_basic_lower > prev_final_lower or 
            prev_close < prev_final_lower):
            df.loc[curr_idx, 'Final_Lower'] = curr_basic_lower
        else:
            df.loc[curr_idx, 'Final_Lower'] = prev_final_lower
            
        # Update current final values after calculation
        curr_final_upper = df.loc[curr_idx, 'Final_Upper'].item()
        curr_final_lower = df.loc[curr_idx, 'Final_Lower'].item()
        
        # Supertrend
        if prev_supertrend == prev_final_upper:
            if curr_close > curr_final_upper:
                df.loc[curr_idx, 'Supertrend'] = curr_final_lower
                df.loc[curr_idx, 'Trend'] = 1
            else:
                df.loc[curr_idx, 'Supertrend'] = curr_final_upper
                df.loc[curr_idx, 'Trend'] = -1
        else:
            if curr_close < curr_final_lower:
                df.loc[curr_idx, 'Supertrend'] = curr_final_upper
                df.loc[curr_idx, 'Trend'] = -1
            else:
                df.loc[curr_idx, 'Supertrend'] = curr_final_lower
                df.loc[curr_idx, 'Trend'] = 1
    
    return df

def fetch_stock_data(symbol, start_date, end_date, interval='1wk'):
    """Fetch stock data from Yahoo Finance"""
    try:
        stock = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        if stock.empty:
            raise Exception(f"No data found for {symbol}")
        return stock
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def analyze_supertrend(symbol, period=9, multiplier=2):
    """Analyze stock using Supertrend indicator"""
    # Calculate date range for 2 years of weekly data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=2*365)
    
    # Fetch data
    df = fetch_stock_data(symbol, start_date, end_date)
    if df is None:
        return
    
    # Calculate Supertrend
    result = get_supertrend(df, period=period, multiplier=multiplier)
    
    # Generate signals
    result['Signal'] = np.where(result['Trend'] == 1, 'BUY', 'SELL')
    
    # Print analysis
    print(f"\nSupertrend Analysis for {symbol}")
    print("=" * 50)
    
    # Print last 10 weeks of data
    recent_data = result.tail(10)
    for idx, row in recent_data.iterrows():
        print(f"\nDate: {idx.strftime('%Y-%m-%d')}")
        print(f"Close: ${row['Close']:.2f}")
        print(f"Supertrend: ${row['Supertrend']:.2f}")
        print(f"Signal: {row['Signal']}")
        if row['Trend'] == 1:
            print("Trend: Uptrend 🔼")
        else:
            print("Trend: Downtrend 🔽")
        print("-" * 30)
    
    return result

def main():
    # Example usage
    symbols = ['AAPL', 'MSFT', 'GOOGL']
    
    results = {}
    for symbol in symbols:
        print(f"\nAnalyzing {symbol}...")
        result = analyze_supertrend(symbol, period=9, multiplier=2)
        
        if result is not None:
            results[symbol] = result
            
            # Calculate current position
            current_data = result.iloc[-1]
            print(f"\nCurrent Status for {symbol}:")
            print(f"Current Close: ${current_data['Close']:.2f}")
            print(f"Current Supertrend: ${current_data['Supertrend']:.2f}")
            print(f"Current Signal: {current_data['Signal']}")
            print(f"Trend Direction: {'Uptrend 🔼' if current_data['Trend'] == 1 else 'Downtrend 🔽'}")
    
    return results

if __name__ == "__main__":
    main()


Analyzing AAPL...


[*********************100%***********************]  1 of 1 completed


Supertrend Analysis for AAPL

Date: 2024-12-16


TypeError: unsupported format string passed to Series.__format__

In [ ]:
#GROK
import yfinance as yf
import pandas as pd
import numpy as np

def supertrend(df, period=9, multiplier=2):
    # Calculate ATR (Average True Range)
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    
    tr = pd.DataFrame({'high_low': high_low, 'high_close': high_close, 'low_close': low_close})
    tr['TR'] = tr.max(axis=1)
    atr = tr['TR'].rolling(window=period).mean()
    
    # Calculate Supertrend
    hl2 = (df['High'] + df['Low']) / 2
    upperband = hl2 + (multiplier * atr)
    lowerband = hl2 - (multiplier * atr)
    
    # Initialize columns
    supertrend = [0] * len(df)
    final_upperband = [0] * len(df)
    final_lowerband = [0] * len(df)
    
    # Calculate Supertrend values
    for i in range(period, len(df)):
        if upperband[i] < final_upperband[i-1] or df['Close'][i-1] > final_upperband[i-1]:
            final_upperband[i] = upperband[i]
        else:
            final_upperband[i] = final_upperband[i-1]
            
        if lowerband[i] > final_lowerband[i-1] or df['Close'][i-1] < final_lowerband[i-1]:
            final_lowerband[i] = lowerband[i]
        else:
            final_lowerband[i] = final_lowerband[i-1]
            
        if (supertrend[i-1] == final_upperband[i-1] and df['Close'][i] <= final_upperband[i]) or \
           (supertrend[i-1] == final_lowerband[i-1] and df['Close'][i] >= final_lowerband[i]):
            supertrend[i] = final_upperband[i]
        elif (supertrend[i-1] == final_upperband[i-1] and df['Close'][i] > final_upperband[i]):
            supertrend[i] = final_lowerband[i]
        elif (supertrend[i-1] == final_lowerband[i-1] and df['Close'][i] < final_lowerband[i]):
            supertrend[i] = final_upperband[i]
        else:
            supertrend[i] = supertrend[i-1]
    
    # Create DataFrame with results
    result = pd.DataFrame({
        'Close': df['Close'],
        'Supertrend': supertrend,
        'UpperBand': final_upperband,
        'LowerBand': final_lowerband
    }, index=df.index)
    
    return result

# Read symbols from CSV file
csv_file = "nse_tickers.csv"  # Replace with your CSV file path
try:
    symbols_df = pd.read_csv(csv_file)
    if 'Symbol' not in symbols_df.columns:
        raise ValueError("CSV file must contain a 'Symbol' column")
    symbols = symbols_df['Symbol'].tolist()
except FileNotFoundError:
    print(f"Error: {csv_file} not found")
    exit()
except ValueError as e:
    print(f"Error: {e}")
    exit()

# Process each symbol
for symbol in symbols:
    try:
        # Fetch stock data from Yahoo Finance with weekly interval
        stock = yf.Ticker(symbol)
        df = stock.history(period="3y", interval="1wk")  # Get 1 year of weekly data
        
        if df.empty:
            print(f"No data found for {symbol}")
            continue
            
        # Calculate Supertrend
        st_result = supertrend(df, period=9, multiplier=2)
        
        # Display the last 5 weeks
        print(f"\nSupertrend (9,2) for {symbol} (Weekly)")
        print(st_result.tail(5))
        
        # Basic interpretation
        last_close = st_result['Close'].iloc[-1]
        last_supertrend = st_result['Supertrend'].iloc[-1]
        
        trend = "Uptrend" if last_close > last_supertrend else "Downtrend"
        
        print(f"\nCurrent Trend: {trend}")
        print(f"Last Weekly Close Price: {last_close:.2f}")
        print(f"Last Supertrend Value: {last_supertrend:.2f}")
        print("-" * 50)
        
        if trend == "Uptrend":
            print(f"{symbol} is in an Uptrend. Consider buying opportunities.")
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        continue


Supertrend (9,2) for ZOMATO.NS (Weekly)
                                Close  Supertrend   UpperBand   LowerBand
Date                                                                     
2025-01-20 00:00:00+05:30  215.600006  288.244449  288.244449  242.854455
2025-01-27 00:00:00+05:30  236.320007  284.183328  284.183328  161.716666
2025-02-03 00:00:00+05:30  233.899994  284.183328  284.183328  175.387782
2025-02-10 00:00:00+05:30  215.990005  280.126664  280.126664  175.387782
2025-02-17 00:00:00+05:30  230.360001  280.126664  280.126664  175.387782

Current Trend: Downtrend
Last Weekly Close Price: 230.36
Last Supertrend Value: 280.13
--------------------------------------------------


/var/folders/ms/04w2p_357g91jjjd814nsyx80000gn/T/ipykernel_65144/3910825305.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if upperband[i] < final_upperband[i-1] or df['Close'][i-1] > final_upperband[i-1]:
/var/folders/ms/04w2p_357g91jjjd814nsyx80000gn/T/ipykernel_65144/3910825305.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  final_upperband[i] = upperband[i]
/var/folders/ms/04w2p_357g91jjjd814nsyx80000gn/T/ipykernel_65144/3910825305.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFr

In [25]:
#ChatGPT
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def atr(df, period=14):
    """Calculate Average True Range (ATR) using EMA."""
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())

    tr = pd.DataFrame({'high_low': high_low, 'high_close': high_close, 'low_close': low_close})
    tr['TR'] = tr.max(axis=1)

    # Use EMA for ATR calculation (more accurate)
    atr = tr['TR'].ewm(span=period, adjust=False).mean()
    return atr

def supertrend(df, period=9, multiplier=2):
    """Calculate Supertrend Indicator"""
    df = df.copy()
    df['ATR'] = atr(df, period)

    # Calculate basic bands
    hl2 = (df['High'] + df['Low']) / 2
    df['UpperBand'] = hl2 + (multiplier * df['ATR'])
    df['LowerBand'] = hl2 - (multiplier * df['ATR'])

    # Supertrend calculation
    df['Supertrend'] = np.nan
    df['FinalUpperBand'] = df['UpperBand']
    df['FinalLowerBand'] = df['LowerBand']

    for i in range(1, len(df)):
        # Upper Band Adjustment
        if df['UpperBand'].iloc[i] < df['FinalUpperBand'].iloc[i-1] or df['Close'].iloc[i-1] > df['FinalUpperBand'].iloc[i-1]:
            df.at[df.index[i], 'FinalUpperBand'] = df['UpperBand'].iloc[i]
        else:
            df.at[df.index[i], 'FinalUpperBand'] = df['FinalUpperBand'].iloc[i-1]

        # Lower Band Adjustment
        if df['LowerBand'].iloc[i] > df['FinalLowerBand'].iloc[i-1] or df['Close'].iloc[i-1] < df['FinalLowerBand'].iloc[i-1]:
            df.at[df.index[i], 'FinalLowerBand'] = df['LowerBand'].iloc[i]
        else:
            df.at[df.index[i], 'FinalLowerBand'] = df['FinalLowerBand'].iloc[i-1]

        # Determine Supertrend
        if np.isnan(df['Supertrend'].iloc[i-1]):
            df.at[df.index[i], 'Supertrend'] = df['FinalLowerBand'].iloc[i] if df['Close'].iloc[i] > df['FinalLowerBand'].iloc[i] else df['FinalUpperBand'].iloc[i]
        elif df['Supertrend'].iloc[i-1] == df['FinalUpperBand'].iloc[i-1] and df['Close'].iloc[i] <= df['FinalUpperBand'].iloc[i]:
            df.at[df.index[i], 'Supertrend'] = df['FinalUpperBand'].iloc[i]
        elif df['Supertrend'].iloc[i-1] == df['FinalUpperBand'].iloc[i-1] and df['Close'].iloc[i] > df['FinalUpperBand'].iloc[i]:
            df.at[df.index[i], 'Supertrend'] = df['FinalLowerBand'].iloc[i]
        elif df['Supertrend'].iloc[i-1] == df['FinalLowerBand'].iloc[i-1] and df['Close'].iloc[i] >= df['FinalLowerBand'].iloc[i]:
            df.at[df.index[i], 'Supertrend'] = df['FinalLowerBand'].iloc[i]
        elif df['Supertrend'].iloc[i-1] == df['FinalLowerBand'].iloc[i-1] and df['Close'].iloc[i] < df['FinalLowerBand'].iloc[i]:
            df.at[df.index[i], 'Supertrend'] = df['FinalUpperBand'].iloc[i]

    return df[['Close', 'Supertrend', 'FinalUpperBand', 'FinalLowerBand']]

# Read symbols from CSV file
csv_file = "nse_tickers.csv"  # Replace with your CSV file path
try:
    symbols_df = pd.read_csv(csv_file)
    if 'Symbol' not in symbols_df.columns:
        raise ValueError("CSV file must contain a 'Symbol' column")
    symbols = symbols_df['Symbol'].tolist()
except FileNotFoundError:
    print(f"Error: {csv_file} not found")
    exit()
except ValueError as e:
    print(f"Error: {e}")
    exit()

# Initialize an empty DataFrame to store summary data
all_summary_df = pd.DataFrame()

# Process each symbol
for symbol in symbols:
    try:
        stock = yf.Ticker(f"{symbol}.NS")
        df = stock.history(period="3y", interval="1wk")

        if df.empty:
            print(f"No data found for {symbol}")
            continue

        st_result = supertrend(df, period=9, multiplier=2)

        # Display last 5 values
        # print(f"\nSupertrend (9,2) for {symbol} (Weekly)")
        # print(st_result.tail(5))

        # Identify trend
        last_close = st_result['Close'].iloc[-1]
        last_supertrend = st_result['Supertrend'].iloc[-1]
        trend = "Uptrend" if last_close > last_supertrend else "Downtrend"
        
        print(f"\nSupertrend (9,2) for {symbol} (Weekly)")
        print(f"\nCurrent Trend: {trend}")
        print(f"Last Weekly Close Price: {last_close:.2f}")
        print(f"Last Supertrend Value: {last_supertrend:.2f}")

        # Append data to a single DataFrame
        summary_data = {
            'Symbol': symbol,
            'Current Trend': trend,
            'Last Weekly Close Price': last_close,
            'Last Supertrend Value': last_supertrend
        }
        all_summary_df = pd.concat([all_summary_df, pd.DataFrame([summary_data])], ignore_index=True)

    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        continue

# Save the combined DataFrame to an Excel file once after processing all symbols
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
excel_filename = f"all_symbols_summary_{timestamp}.xlsx"
all_summary_df.to_excel(excel_filename, index=False)
print(f"Summary saved to {excel_filename}")


Supertrend (9,2) for 360ONE (Weekly)

Current Trend: Downtrend
Last Weekly Close Price: 981.25
Last Supertrend Value: 1174.39

Supertrend (9,2) for 3MINDIA (Weekly)

Current Trend: Downtrend
Last Weekly Close Price: 27344.10
Last Supertrend Value: 30537.77

Supertrend (9,2) for ABB (Weekly)

Current Trend: Downtrend
Last Weekly Close Price: 5291.40
Last Supertrend Value: 6319.30
Summary saved to all_symbols_summary_20250223_115022.xlsx
